In [3]:
import numpy as np

x = np.random.rand(2000, 2000)
print(x)

[[0.74274912 0.68354463 0.53984386 ... 0.37261657 0.50623772 0.52102898]
 [0.8065987  0.22152095 0.94631063 ... 0.8896629  0.39416147 0.31897523]
 [0.81070736 0.06077232 0.07030569 ... 0.48668678 0.95820186 0.22619086]
 ...
 [0.69912444 0.973821   0.55607617 ... 0.07932041 0.56458864 0.02349056]
 [0.10156827 0.36752652 0.87417821 ... 0.80616384 0.11749962 0.09186798]
 [0.97643735 0.93127403 0.53943877 ... 0.47635364 0.41610516 0.9589991 ]]


In [5]:
import jax.numpy as jnp

y = jnp.array(x) # device array
print(y)
print('*'*30)
print(type(y))

[[0.7427491  0.68354464 0.53984386 ... 0.37261656 0.50623775 0.521029  ]
 [0.8065987  0.22152095 0.94631064 ... 0.8896629  0.39416146 0.31897524]
 [0.8107074  0.06077232 0.07030569 ... 0.48668677 0.9582019  0.22619087]
 ...
 [0.69912446 0.973821   0.55607617 ... 0.07932041 0.56458867 0.02349056]
 [0.10156827 0.36752653 0.87417823 ... 0.80616385 0.11749962 0.09186798]
 [0.97643733 0.93127406 0.5394388  ... 0.47635365 0.41610515 0.9589991 ]]
******************************
<class 'jaxlib.xla_extension.ArrayImpl'>
